In [ ]:
pip install finpie

In [ ]:
pip install yahoofinancials

In [ ]:
pip install yahoo_fin

In [ ]:
pip install jupyter_dash

In [ ]:
pip install dash

In [ ]:
pip install dash_core_components

In [ ]:
pip install dash_html_components

In [ ]:
pip install dash_table

In [ ]:
from yahoofinancials import YahooFinancials

import yahoo_fin.stock_info as si
import finpie

import csv
import numpy as np
import pandas as pd

## Dash Code

In [10]:
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc

from dash_table import DataTable
from dash_table import FormatTemplate

from dash.dependencies import Input, Output

import plotly.express as px
from datetime import datetime

#from yahoofinancials import YahooFinancials

#import yahoo_fin.stock_info as si
#import finpie

import csv
import numpy as np
import pandas as pd

#src0 = 'https://raw.githubusercontent.com/Bayan2019/TCC/main/list_comp.csv'
src1 = 'https://raw.githubusercontent.com/Bayan2019/TCC/main/list_comp_fs_yahoo.csv'

yahoo_logo = 'https://raw.githubusercontent.com//Bayan2019/TCC/main/Yahoo-Finance-3.png'
tcc_logo = 'https://raw.githubusercontent.com//Bayan2019/TCC/main/Logo_Curve_BG-Blue1.png'
jci_logo = 'https://raw.githubusercontent.com//Bayan2019/TCC/main/jc2.png'

CaaSWalletIPhone = 'https://raw.githubusercontent.com//Bayan2019/TCC/main/CaaSWallet2_2.png'

url_annual_json = 'https://raw.githubusercontent.com/Bayan2019/TCC/main/DB_focus_annual.json'
url_quartal_json = 'https://raw.githubusercontent.com/Bayan2019/TCC/main/DB_focus_quartal.json'

url_annual_csv = 'https://raw.githubusercontent.com/Bayan2019/TCC/main/fs_annual.csv'
url_quartal_csv = 'https://raw.githubusercontent.com/Bayan2019/TCC/main/fs_quartal.csv'

tickers = pd.read_csv(src1)

set_df_annual = pd.read_json(url_annual_json)
set_df_quartal = pd.read_json(url_quartal_json)

big_df_annual = pd.read_csv(url_annual_csv)
big_df_quartal = pd.read_csv(url_quartal_csv)

Big_df_annual = big_df_annual
Big_df_annual = Big_df_annual.drop(['Unnamed: 0', 'ticker', 'date', 'company_name', 'sector', 'specific'], 
                                   axis=1)

All_columns = list(Big_df_annual.columns)

options_period = [{'label':'Annual', 'value':'annual'}, {'label':'Quartal', 'value':'quarterly'}]
options_ticker = [{'label': tickers.loc[i, 'ticker']+' -- '+tickers.loc[i, 'company_name'], 
                            'value':tickers.loc[i, 'ticker']} for i in tickers.index]

#def get_inf_cols(data_frame):
#    all_columns = list(data_frame.columns)
#    columns = all_columns
#    
#    data_frame['NaN']=np.nan
#    
#    for col in all_columns:
#        if data_frame[col].equals(data_frame['NaN']): del columns[columns.index(col)]
#    
#    return columns

def get_title(ticker):
    df = tickers.set_index('ticker')
    title = df.loc[ticker, 'company_name']
    
    return title

def plot_style(paper_bg='black', plot_bg='rgb(82, 82, 82)', font='violet'):
    plot_style = {'paper_bgcolor':paper_bg, 'plot_bgcolor':plot_bg,
                               'font': {'color':font}}
    return plot_style

# Create the Dash app
app = JupyterDash(__name__)

app.layout = html.Div([
 #html.Img(src=jci_logo, style={'width':'300px', 'display':'inline-block'}),
 html.Img(src=yahoo_logo, style={'width':'250px', 'display':'inline-block', 'margin':'0px 60px 0px 60px'}),   
 html.H1('Financial Statement', style={'color':'rgb(191, 62, 255)', 'fontSize': 70, 
                                       'display':'inline-block'}),
 html.Img(src=yahoo_logo, style={'width':'250px', 'display':'inline-block', 'margin':'0px 60px 0px 60px'}),
 #html.Img(src=jci_logo, style={'width':'300px', 'display':'inline-block'}),
 html.Br(),
# TreeMap   
 html.Div(
  children = [
  html.Div(
   children = [  
   html.H3('Period'),
   dcc.Dropdown(id='period_treemap', options = options_period, value = 'annual',
        style={'width':'100px', 'margin':'0 auto', 'color':'blue', 'verticalAlign':'top'})],
   style={'display':'inline-block', 'margin':'0px 10px 0px 10px'}),
  html.Div(
   children = [   
   html.H3('Organizations to Exclude'),
   dcc.Dropdown(id='tickers_list_treemap', options=tickers.ticker.unique(), value=[], multi=True)],
   style={'display':'inline-block', 'margin':'0px 10px 0px 10px'})], 
  style={'width':'1400px', 'height':'200px', 'display':'inline-block', 'border':'1px solid black'}),   
 dcc.Graph(id='treemap'),
# Bar plot for comparison different companies    
 html.Div(
  children = [
  html.Div(
   children = [
   html.H3('Organizations to Include'),
   dcc.Dropdown(id='tickers_list_bar0', options=tickers.ticker.unique(),
                value=['ELY', 'JCI', 'FLEX'], multi=True),
   html.H3('Period'),
   dcc.Dropdown(id='period_bar0', options = options_period, value = 'annual',
        style={'width':'100px', 'margin':'0 auto', 'color':'blue', 'verticalAlign':'top'})       
   html.H3('Columns to Include'),
   dcc.Dropdown(id='columns_list_bar0', options=All_columns, value=['inventory', 'cash'], multi=True)],
   style={'width':'250px', 'height':'450px', 'display':'inline-block',
    'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
  html.Div(
   children = [dcc.Graph(id='bar_fig0')], 
   style={'width':'900px', 'height':'450px','display':'inline-block'})],
  style={'width':'1400px', 'height':'450px', 'display':'inline-block', 'border':'1px solid black'}),
# choosing Particular company to consider    
 html.Div(
  children = [   
  html.H3('Organization'),  
  dcc.Dropdown(id='ticker_for_only_annual', 
    options = options_ticker, style={'width':'700px', 'margin':'0 auto', 'color':'blue'})],
   style={'display':'inline-block', 'margin':'0px 15px 0px 15px'}), 
 html.Br(),   
 html.Div(
  children = [
  html.Span(id='information', style={'width':'800px', 'display':'inline-block',
                                     'vertical-align':'top', 'margin':'40px 30px 0px 30px'}), 
  html.H2(html.Table([
        html.Tr([html.Td(['Profit Margin']), html.Td(id='ProfitMargin')]),
        html.Tr([html.Td(['Operating Margin (ttm)']), html.Td(id='OperatingMargin')]),
        html.Tr([html.Td(['Return on Assets (ttm)']), html.Td(id='ReturnOnAssets')]),
        html.Tr([html.Td(['Audit Risk']), html.Td(id='AR')]),
        html.Tr([html.Td(['Total Cash Per Share (mrq)']), html.Td(id='TCPSh')]),
        html.Tr([html.Td(['Current Ratio (mrq)']), html.Td(id='CR')]),
    ]), style={'width':'400px','font_size': '76px', 'display':'inline-block', 'vertical-align':'top', 
               'margin':'40px 30px 0px 30px'})
  ],
  style={'display':'inline-block', 'vertical-align':'center'}),
# Here probably will be the annual plot 
    
 html.Div(
  children = [
  html.Div(
   children = [  
       
   html.H3('y-axis'),
   dcc.Dropdown(id='column_y', 
                style={'width':'250px', 'margin':'0 auto', 'color':'blue', 'verticalAlign':'top'})],
   style={'display':'inline-block', 'margin':'0px 10px 0px 10px'}),
  html.Div(
   children = [   
   html.H3('ColorMap'),
   dcc.Dropdown(id='column_c', style={'width':'250px', 'color':'blue', 'verticalAlign':'top'})],
   style={'display':'inline-block', 'margin':'0px 10px 0px 10px'}),
  html.Div(
   children = [  
   html.H3('Radius'),
   dcc.Dropdown(id='column_r', 
                style={'width':'250px', 'margin':'0 auto', 'color':'blue', 'vertical-align':'top'})],
   style={'display':'inline-block', 'margin':'0px 10px 0px 10px'})], 
  style={'width':'1400px', 'height':'120px', 'display':'inline-block', 'border':'1px solid black'}),
 html.Br(),
 html.Div(
  children = [
  html.Img(src=CaaSWalletIPhone, style={'width':'500px', 'height':'700px', 'display':'inline-block'}), 
  html.Div(
   children = [    
   dcc.Graph(id='scatter_fig', style={'width':'700px'}),
   dcc.Graph(id='bar_fig1', style={'width':'800px'})],
   style={'width':'850px', 'height':'700px', 'display':'inline-block'})],
  style={'display':'inline-block', 'vertical-align':'center'}),
 dcc.Graph(id='bar_fig2'),
 html.Br(),
 html.Div(
  children = [    
   dcc.Graph(id='inv_rev_fig', style={'width':'600px', 'height':'400px', 'display':'inline-block'}),
   dcc.Graph(id='inv_cash_fig', style={'width':'600px', 'height':'400px', 'display':'inline-block'})],
   style={'width':'1200px', 'height':'400px', 'display':'inline-block'}),
 html.Div(
  children = [    
   dcc.Graph(id='gross_pr_inv_fig', style={'width':'600px', 'height':'400px', 'display':'inline-block'}),
   dcc.Graph(id='gross_pr_rev_fig', style={'width':'600px', 'height':'400px', 'display':'inline-block'})],
   style={'width':'1200px', 'height':'400px', 'display':'inline-block'}), 
 html.Br(),   
 html.Div(
  children = [
  html.Div(
   children = [
   html.H3('First Column'),
   dcc.Dropdown(id='column_1', style={'width':'250px', 'color':'blue'}),
   html.H3('Second Column'),
   dcc.Dropdown(id='column_2', style={'width':'250px', 'color':'blue'}),
   html.H3('Third Column'),
   dcc.Dropdown(id='column_3', style={'width':'250px', 'color':'blue'})],
   style={'width':'250px', 'height':'450px', 'display':'inline-block',
    'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
  html.Div(
   children = [   
   dcc.Graph(id='line_polar_fig')], 
   style={'width':'900px', 'height':'450px','display':'inline-block'})],
  style={'width':'1400px', 'height':'450px', 'display':'inline-block', 'border':'1px solid black'}),
 html.Br(),
 html.Div(
  children = [
  html.Div(
   children = [
   html.H3('Organizations to Exclude'),
   dcc.Dropdown(id='tickers_list3', options=tickers.ticker.unique(), value=[], multi=True)],
   style={'width':'250px', 'height':'450px', 'display':'inline-block',
    'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
  html.Div(
   children = [   
   dcc.Graph(id='sunburst')], 
   style={'width':'900px', 'height':'450px','display':'inline-block'})],
  style={'width':'1400px', 'height':'450px', 'display':'inline-block', 'border':'1px solid black'}),       
 html.Br(),
 html.Img(src=tcc_logo, style={'width':'250px', 'display':'inline-block', 'margin':'0px 60px 0px 60px'}),   
 html.Span(
  children = [
  f"Prepared on {(datetime.now().date()).strftime('%B %d, %Y')}",
  html.Br(),
  'by ', html.B('Bayan Saparbayeva'),
  html.Br(),
  html.B('Alexandre Courtois Auberger'),
  html.Br(),    
  'and ', html.B('Daksha Daniel'),
  html.Br(),
  html.Br(),],
  style={'text-align':'center', 'font-size':22, 'display':'inline-block'}),
 html.Img(src=tcc_logo, style={'width':'250px', 'display':'inline-block', 'margin':'0px 60px 0px 60px'})],
 style={'text-align':'center', 'display':'inline-block', 'width':'100%',
        'background-color':'black', 'color':'deeppink'})
    
@app.callback(
 Output(component_id='treemap', component_property='figure'),
 Input(component_id='period', component_property='value'),
 Input(component_id='tickers_list1', component_property='value')   
)
    
def update_treemap(per, ticks):       
    if per=='annual': df = big_df_annual
    if per=='quarterly': df = big_df_quartal
        
    tickers_list = ticks    
    
    df['inventory_to_assets'] = df['inventory'].div(df['totalAssets'])
    df['date'] = pd.to_datetime(df['date'])
        
    treemap_fig = px.treemap(df[~df.ticker.isin(tickers_list)], 
                  path=[px.Constant('all sectors'), 'sector', 'specific', 'ticker', 'date'], 
                  values='inventory_to_assets',
                  color='inventory', hover_data=['company_name'], color_continuous_scale='RdYlGn_r')
    treemap_fig.update_layout(plot_style())
 
    return treemap_fig
    
@app.callback(
 Output(component_id='sunburst', component_property='figure'),
 Input(component_id='period', component_property='value'),
 Input(component_id='tickers_list3', component_property='value')    
)
    
def update_sunburst(per, ticks):       
    df = big_df_annual
    if per=='quarterly': df = big_df_quartal
        
    tickers_list = ticks     
    
    df['inventory_to_assets'] = df['inventory'].div(df['totalAssets'])
        
    sunburst_fig = px.sunburst(df[~df.ticker.isin(tickers_list)], 
                  path=['sector', 'specific', 'ticker', 'date'], values='inventory_to_assets',
                  color='inventory', hover_data=['company_name'], color_continuous_scale='RdYlGn_r')
    sunburst_fig.update_layout(plot_style())
 
    return sunburst_fig

@app.callback(
 Output(component_id='bar_fig0', component_property='figure'),
 Input(component_id='period', component_property='value'),
 Input(component_id='tickers_list2', component_property='value'),
 Input(component_id='columns_list2', component_property='value'),   
)

def update_bar0(per, ticks, cols):
    tickers_list = ticks
    
    df = big_df_annual
    if per=='quarterly': df = big_df_quartal
        
    columns_list = cols
    
    df_dropped = df.drop(['Unnamed: 0', 'company_name', 'sector', 'specific'], axis=1)

    df_melt = pd.melt(df_dropped, id_vars=['date', 'ticker'])

    df_melt['date'] = pd.to_datetime(df_melt['date'])
    df_melt['value'] = df_melt['value'].astype('float')

    df_melt_cols = df_melt[df_melt.variable.isin(columns_list)]

    df_melt_cols_ticks = df_melt_cols[df_melt_cols.ticker.isin(tickers_list)]
        
    bar_fig0 = px.bar(df_melt_cols_ticks, y='value', color='variable', 
                      x='date', barmode="group", facet_row="ticker", custom_data=['date'])
    bar_fig0.update_layout({'paper_bgcolor':'black', 'plot_bgcolor':'rgb(82, 82, 82)',
                               'font': {'color':'violet'}})
 
    return bar_fig0

#html.Td(['Profit Margin']), html.Td(id='ProfitMargin')]),
#html.Tr([html.Td(['Operating Margin (ttm)']), html.Td(id='OperatingMargin')]),
#html.Tr([html.Td(['Return on Assets (ttm)']), html.Td(id='ReturnOnAssets')]),
#html.Tr([html.Td(['Audit Risk']), html.Td(id='AR')]),
#html.Tr([html.Td(['Total Cash Per Share (mrq)']), html.Td(id='TCPSh')]),
#html.Tr([html.Td(['Current Ratio (mrq)']), html.Td(id='CR')]),

@app.callback(
 Output(component_id='information', component_property='children'),
 Output(component_id='ProfitMargin', component_property='children'),
 Output(component_id='OperatingMargin', component_property='children'),
 Output(component_id='ReturnOnAssets', component_property='children'),
 Output(component_id='AR', component_property='children'),   
 Output(component_id='TCPSh', component_property='children'),
 Output(component_id='CR', component_property='children'), 
 Input(component_id='ticker', component_property='value')
)

def update_informatiol(tick):
    ticker='ELY'
    if tick: ticker=tick
    
    information_text = si.get_company_info(ticker).loc['longBusinessSummary', 'Value']
    PM = si.get_stats(ticker).set_index('Attribute').loc['Profit Margin', 'Value']
    OM = si.get_stats(ticker).set_index('Attribute').loc['Operating Margin (ttm)', 'Value']
    RA = si.get_stats(ticker).set_index('Attribute').loc['Operating Margin (ttm)', 'Value']
    AR = si.get_company_info(ticker).loc['auditRisk', 'Value']
    TCPSh = finpie.Fundamentals(ticker, 'yahoo').key_metrics()['total_cash_per_share_(mrq)'][0]
    CR = finpie.Fundamentals(ticker, 'yahoo').key_metrics()['current_ratio_(mrq)'][0]
    
    return information_text, PM, OM, RA, AR, TCPSh, CR

@app.callback(
 Output(component_id='column_y', component_property='options'),
 Output(component_id='column_c', component_property='options'),
 Output(component_id='column_1', component_property='options'),   
 Output(component_id='column_r', component_property='options'),
 Input(component_id='ticker', component_property='value'),
 Input(component_id='period', component_property='value')    
)

def update_columns(tick, per):
    ticker = 'ELY'
    if tick: ticker=tick
    
    set_df = set_df_annual
    if per=='quarterly': set_df = set_df_quartal
    
    df = pd.DataFrame(dict(zip(set_df[ticker].index, set_df[ticker].values))).T.dropna().T
    
    #df.set_index('date', inplace=True)
    #df.index = pd.to_datetime(df.index)
    
    columns1 = list(df.columns)
    
    del columns1[columns1.index('date')]
    
    column_options = [{'label':x, 'value':x} for x in columns1]
    
    positive_columns = []

    for col in columns1:
        if (df[col] > 0.1).all(): positive_columns.append(col)
            
    column_r_options = [{'label':x, 'value':x} for x in positive_columns]
    
    return column_options, column_options, column_options, column_r_options

@app.callback(
 Output(component_id='scatter_fig', component_property='figure'),
 Input(component_id='ticker', component_property='value'),
 Input(component_id='period', component_property='value'),
 Input(component_id='column_y', component_property='value'),
 Input(component_id='column_c', component_property='value'),
 Input(component_id='column_r', component_property='value')   
)

def update_scatter(tick, per, column_name_y, column_name_c, column_name_r):
    ticker = 'ELY'
    if tick: ticker=tick
        
    col_name_y = 'totalAssets'
    if column_name_y: col_name_y = column_name_y
        
    col_name_c = 'totalLiab'
    if column_name_c: col_name_c = column_name_c

    col_name_r = 'totalStockholderEquity'
    if column_name_r: col_name_r = column_name_r
        
    set_df = set_df_annual
    if per=='quarterly': set_df = set_df_quartal
    
    df = pd.DataFrame(dict(zip(set_df[ticker].index, set_df[ticker].values))).T.dropna().T
    
    df['date'] = pd.to_datetime(df['date'])
    
    columns1 = list(df.columns)
    
    del columns1[columns1.index('date')]
    
    df[columns1] = df[columns1].astype('float')
        
    scatter_fig = px.scatter(df, x='date', y=col_name_y, size=col_name_r, color=col_name_c,
            custom_data=['date'], size_max=50, height=360,
            title=get_title(ticker), color_continuous_scale='RdYlGn_r')
    scatter_fig.update_layout(plot_style())
 
    return scatter_fig

@app.callback(
 Output(component_id='bar_fig1', component_property='figure'),
 Input(component_id='ticker', component_property='value'),
 Input(component_id='period', component_property='value')
)

def update_bar1(tick, per):
    ticker = 'ELY'
    if tick: ticker=tick
    
    set_df = set_df_annual
    if per=='quarterly': set_df = set_df_quartal
    
    df = pd.DataFrame(dict(zip(set_df[ticker].index, set_df[ticker].values))).T.dropna().T
    
    df['date'] = pd.to_datetime(df['date'])
        
    df_melt = pd.melt(df, id_vars=['date'])    
    
    #fs_melt['date'] = fs_melt['date'].astype('str')
    df_melt['value'] = df_melt['value'].astype('float')
            
    bar_fig1 = px.bar(df_melt[df_melt.variable.isin(['inventory',
        'cash', 'totalRevenue', 'ebit', 'shortLongTermDebt'])], 
                      height=330,
        y='value', color='variable', x='date', barmode="group", custom_data=['date'])
    bar_fig1.update_layout({'paper_bgcolor':'black', 'plot_bgcolor':'rgb(82, 82, 82)',
                               'font': {'color':'violet'}})
 
    return bar_fig1

@app.callback(
 Output(component_id='bar_fig2', component_property='figure'),
 Input(component_id='ticker', component_property='value'),
 Input(component_id='period', component_property='value'),
 Input(component_id='bar_fig1', component_property='clickData')   
)

def update_bar2(tick, per, clickData):
    ticker = 'ELY'
    if tick: ticker=tick
    
    set_df = set_df_annual
    if per=='quarterly': set_df = set_df_quartal
    
    df = pd.DataFrame(dict(zip(set_df[ticker].index, set_df[ticker].values))).T.dropna().T
    
    df['date'] = pd.to_datetime(df['date'])
    
    df.set_index('date', inplace=True)
        
    cdate = df.index[0]
    if clickData: cdate = pd.to_datetime(clickData['points'][0]['customdata'][0])  
        
    ser = df.loc[cdate, :].astype('float')
        
    bar_fig2 = px.bar(ser, height=600, title=get_title(ticker))
    bar_fig2.update_layout(plot_style())
 
    return bar_fig2

#   dcc.Graph(id='inv_rev_fig', style={'width':'600px', 'display':'inline-block'}),
#   dcc.Graph(id='inv_cash_fig', style={'width':'600px', 'display':'inline-block'})],
#   dcc.Graph(id='gross_pr_inv_fig', style={'width':'600px', 'display':'inline-block'}),
#   dcc.Graph(id='gross_pr_rev_fig', style={'width':'600px', 'display':'inline-block'})],


@app.callback(
 Output(component_id='inv_rev_fig', component_property='figure'),
 Output(component_id='inv_cash_fig', component_property='figure'),
 Output(component_id='gross_pr_inv_fig', component_property='figure'),   
 Output(component_id='gross_pr_rev_fig', component_property='figure'),
 Input(component_id='ticker', component_property='value'),
 Input(component_id='period', component_property='value')    
)

def update_scatter_figures(tick, per):
    ticker = 'ELY'
    if tick: ticker=tick
    
    set_df = set_df_annual
    if per=='quarterly': set_df = set_df_quartal
    
    df = pd.DataFrame(dict(zip(set_df[ticker].index, set_df[ticker].values)))
    
    df['date'] = pd.to_datetime(df['date']) 
    
    columns1 = list(df.columns)
    
    del columns1[columns1.index('date')]
    
    df[columns1] = df[columns1].astype('float')
    
    df_dropna = df[['cash', 'inventory', 'totalRevenue', 'grossProfit', 'date']].dropna()
        
    inv_rev_fig = px.scatter(df_dropna, color='cash', 
                             size='grossProfit', size_max=50,
                             y='inventory', x='totalRevenue', 
                             color_continuous_scale='RdYlGn_r', hover_data=['date'])
    inv_rev_fig.update_layout(plot_style())
    
    inv_cash_fig = px.scatter(df_dropna, color='totalRevenue', 
                             size='grossProfit', size_max=50,
                             y='inventory', x='cash', 
                             color_continuous_scale='RdYlGn_r', hover_data=['date'])
    inv_cash_fig.update_layout(plot_style())
    
    gross_pr_inv_fig = px.scatter(df_dropna, color='cash', 
                             size='totalRevenue', size_max=50,
                             y='grossProfit', x='inventory', 
                             color_continuous_scale='RdYlGn_r', hover_data=['date'])
    gross_pr_inv_fig.update_layout(plot_style())
    
    gross_pr_rev_fig = px.scatter(df_dropna, color='inventory', 
                             size='cash', size_max=50,
                             y='grossProfit', x='totalRevenue', 
                             color_continuous_scale='RdYlGn_r', hover_data=['date'])
    
    gross_pr_rev_fig.update_layout(plot_style())
 
    return inv_rev_fig, inv_cash_fig, gross_pr_inv_fig, gross_pr_rev_fig

@app.callback(
 Output(component_id='column_2', component_property='options'),
 Input(component_id='ticker', component_property='value'),
 Input(component_id='period', component_property='value'),
 Input(component_id='column_1', component_property='value')   
)

def update_columns2(tick, per, column_name_1):
    ticker = 'ELY'
    if tick: ticker=tick
    
    set_df = set_df_annual
    if per=='quarterly': set_df = set_df_quartal
    
    df = pd.DataFrame(dict(zip(set_df[ticker].index, set_df[ticker].values))).T.dropna().T
        
    col_name_1 = 'totalAssets'
    if column_name_1: col_name_1 = column_name_1
    
    columns2 = list(df.columns)
    del columns2[columns2.index('date')]
    del columns2[columns2.index(col_name_1)]
    
            
    column_2_options = [{'label':x, 'value':x} for x in columns2]
    
    return column_2_options

@app.callback(
 Output(component_id='column_3', component_property='options'),
 Input(component_id='ticker', component_property='value'),
 Input(component_id='period', component_property='value'),
 Input(component_id='column_1', component_property='value'),
 Input(component_id='column_2', component_property='value')   
)

def update_columns3(tick, per, column_name_1, column_name_2):
    ticker = 'ELY'
    if tick: ticker=tick
    
    set_df = set_df_annual
    if per=='quarterly': set_df = set_df_quartal
    
    df = pd.DataFrame(dict(zip(set_df[ticker].index, set_df[ticker].values))).T.dropna().T
        
    col_name_1 = 'totalAssets'
    if column_name_1: col_name_1 = column_name_1
        
    col_name_2 = 'totalLiab'
    if column_name_2: col_name_2 = column_name_2
    
    columns3 = list(df.columns)
    del columns3[columns3.index('date')]
    del columns3[columns3.index(col_name_1)]
    del columns3[columns3.index(col_name_2)]
    
            
    column_3_options = [{'label':x, 'value':x} for x in columns3]
    
    return column_3_options

@app.callback(
 Output(component_id='line_polar_fig', component_property='figure'),
 Input(component_id='ticker', component_property='value'),
 Input(component_id='period', component_property='value'),   
 Input(component_id='column_1', component_property='value'),
 Input(component_id='column_2', component_property='value'),
 Input(component_id='column_3', component_property='value')   
)

def update_plot4(tick, per, column_name_1, column_name_2, column_name_3):
    ticker = 'ELY'
    if tick: ticker=tick
        
    col_name_1 = 'totalAssets'
    if column_name_1: col_name_1 = column_name_1
        
    col_name_2 = 'totalLiab'
    if column_name_2: col_name_2 = column_name_2

    col_name_3 = 'totalStockholderEquity'
    if column_name_3: col_name_3 = column_name_3
        
    set_df = set_df_annual
    if per=='quarterly': set_df = set_df_quartal
    
    df = pd.DataFrame(dict(zip(set_df[ticker].index, set_df[ticker].values))).T.dropna().T
    
    df_melt = pd.melt(df, id_vars=['date'])
    
    #df_melt['date'] = df_melt['date'].astype('str')
    df_melt['value'] = df_melt['value'].astype('float')
        
    line_polar_fig = px.line_polar(df_melt[df_melt.variable.isin([col_name_1, col_name_2, col_name_3])], 
        r='value', theta='variable', color='date', line_close=True,
        title=get_title(ticker))
    line_polar_fig.update_layout(plot_style())
 
    return line_polar_fig

if __name__ == '__main__':
    app.run_server(port=8050, debug=True)

Dash app running on http://127.0.0.1:8050/


/Users/bayan/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/bayan/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/bayan/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/bayan/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/bayan/opt/anaconda3/lib/python3.9/site-packages/yahoo_fin/stock_info.py:336: FutureWarning:

The frame.append method 

/Users/bayan/opt/anaconda3/lib/python3.9/site-packages/yahoo_fin/stock_info.py:336: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/bayan/opt/anaconda3/lib/python3.9/site-packages/yahoo_fin/stock_info.py:336: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/bayan/opt/anaconda3/lib/python3.9/site-packages/yahoo_fin/stock_info.py:336: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/bayan/opt/anaconda3/lib/python3.9/site-packages/yahoo_fin/stock_info.py:336: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/bayan/opt/anaconda3/lib/python3.9/site-packages/yahoo_fin/stock_info.py:336: FutureWarning:

The frame.append method 